In [80]:
schema = """
{
  "type": "object",
  "properties": {
    "global_time_cost": {"type":  "number"},
    "global_material_modifier": {"type":  "number"},
    "primer_min_anneal": {"type":  "number", "minimum": 5, "maximum":  99},
    "_primer_min_span": {"type": "number"},
    "primer_efficiency": {
      "type":  "object",
      "properties": {
        "data": {
          "type": "array",
          "items": {
            "type": "array",
            "minItems": 3,
            "maxItems": 3
          }
        },
        "columns": {
          "type": "array",
          "items": {"type": "string"},
          "uniqueItems": true,
          "minItems": 3,
          "maxItems": 3
        }
      },
      "required": ["data", "columns"],
      "additionalProperties": false
    },
    "primer_cost": {
      "type":  "object",
      "properties": {
        "data": {
          "type": "array",
          "items": {
            "type": "array",
            "contains":  {"type":  "number"},
            "minItems": 6,
            "maxItems": 6
          }
        },
        "columns": {
          "type": "array",
          "items": {"type": "string"},
          "uniqueItems": true,
          "minItems": 6,
          "maxItems": 6
        }
      },
      "required": ["data", "columns"],
      "additionalProperties": false
    },
    "synthesis_cost": {
      "type": "object",
      "properties": {
        "data": {
          "type": "array",
          "items": {
            "type": "array",
            "minItems": 4,
            "maxItems": 4
          }
        },
        "columns": {
          "type": "array",
          "items": {"type":  "string"},
          "uniqueItems": true,
          "minItems": 4,
          "maxItems": 4
        }
      },
      "additionalProperties": false,
      "required": ["data", "columns"]
    },
    "_synthesis_step_size": {"type":  "number"},
    "_synthesis_left_span_range": {
      "type": "array",
      "items": [
        {"type":  "number"},
        {"type":  "number"}
      ],
      "additionalItems": false
    }
  },
  "minProperties": 9,
  "maxProperties": 9,
  "additionalProperties": false
}
"""

In [85]:
params = {
    'global_time_cost': 10.0,
    'global_material_modifier': 1.0,
    'primer_min_anneal': 16,
    '_primer_min_span': -300,
    'primer_efficiency': {
        'data': [
            [0, 10, 0.0],
            [10, 15, 0.3],
            [15, 20, 0.6],
            [20, 30, 0.8],
            [30, 40, 0.9],
            [40, 50, 0.8],
            [50, 100, 0.75],
            [100, 120, 0.5],
            [120, 150, 0.3],
            [150, 250, 0.1],
            [250, 300, 0.0],
        ],
        'columns': ['min', 'max', 'efficiency']
    },
    'primer_cost': {
        'data':  [
            ["IDTPrimer", 16.0, 60.0, 0.0, 0.15, 1.5],
            ["IDTUltramer", 45.0, 200.0, 0.0, 0.40, 1.5],
        ],
        'columns': ["name", "min", "max", "base cost", "cost per bp", "time (days)"]
    },
    'synthesis_cost': {
        'data': [
            [0, 1, 0, 0],
            [1, 100, 'inf', 'inf'],
            [100, 500, 89.0, 3.0],
            [500, 750, 129.0, 3.0],
            [750, 1000, 149.0, 4.0],
            [1000, 1250, 209.0, 7.0],
            [1250, 1500, 249.0, 7.0],
            [1500, 1750, 289.0, 7.0],
            [1750, 2000, 329.0, 7.0],
            [2000, 2250, 399.0, 7.0],
        ],
        'columns': ["min", "max", "base", "time"]
    },
    '_synthesis_step_size': 10.1,
    '_synthesis_left_span_range': [-500, 500]
}

In [102]:
import pprint

x = pprint.pformat(params).replace('\'', '"').replace('True', 'true').replace('False', 'false')
print(x)

{"_primer_min_span": -300,
 "_synthesis_left_span_range": [-500, 500],
 "_synthesis_step_size": 10.1,
 "global_material_modifier": 1.0,
 "global_time_cost": 10.0,
 "primer_cost": {"columns": ["name",
                             "min",
                             "max",
                             "base cost",
                             "cost per bp",
                             "time (days)"],
                 "data": [["IDTPrimer", 16.0, 60.0, 0.0, 0.15, 1.5],
                          ["IDTUltramer", 45.0, 200.0, 0.0, 0.4, 1.5]]},
 "primer_efficiency": {"columns": ["min", "max", "efficiency"],
                       "data": [[0, 10, 0.0],
                                [10, 15, 0.3],
                                [15, 20, 0.6],
                                [20, 30, 0.8],
                                [30, 40, 0.9],
                                [40, 50, 0.8],
                                [50, 100, 0.75],
                                [100, 120, 0.5],
            

In [49]:
import numpy as np

def add_special_primer_case(params):
    primer_cost_data = params['primer_cost']['data']
    primer_cost_columns = params['primer_cost']['columns']
    i = primer_cost_columns.index('min')
    j = primer_cost_columns.index('max')
    k = primer_cost_columns.index('name')

    new = [0] * len(primer_cost_columns)

    new[k] = 'Non-primer (special case)'
    new[i] = params['primer_min_anneal']
    new[j] = params['primer_min_anneal'] + 1

    primer_cost_data.insert(0, new)
    
def replace_inf(params):
    
    if isinstance(params, list):
        for i, p in enumerate(params):
            if p == 'inf':
                params[i] = np.inf
        for p in params:
            replace_inf(p)
    elif isinstance(params, dict):

        for k, v in params.items():
            if v == 'inf':
                params[k] = np.inf
            else:
                replace_inf(v)
    
    
add_special_primer_case(params)
replace_inf(params)

In [45]:
from dasi.cost import PrimerCostModel, SynthesisCostModel

In [46]:
PrimerCostModel?

Init signature:
PrimerCostModel(
    pdf: pandas.core.frame.DataFrame,
    edf: pandas.core.frame.DataFrame,
    min_anneal: int,
    time_cost: float,
    material_mod: float,
    min_span: int,
)
Docstring:      Abstract base class for building a cost model.
Init docstring:
Initialize the cost model for primer designs.

:param pdf: a pandas DataFrame representing costs for primers.
            Example parameters found at
            :attr:`dasi.cost.params.PrimerParams.primer_df`
:param edf: a pandas DataFrame representing efficiencies for spans.
            Example parameters found at
            :attr:`dasi.cost.params.PrimerParams.eff_df`
:param min_anneal: the number of bases to consider for annealing a primer.
:param time_cost: the cost to produce a primer
:param material_mod:
:param min_span:
File:           ~/Documents/GitHub/DASi-DNA-Design/dasi/cost/__init__.py
Type:           ABCMeta
Subclasses:     


In [39]:
pd.DataFrame(params['primer_cost']['data'])

,0,1,2,3,4,5
0,Non-primer (special case),16.0,17.0,0.0,0.00,0.0
1,IDTPrimer,16.0,60.0,0.0,0.15,1.5
2,IDTUltramer,45.0,200.0,0.0,0.40,1.5


In [42]:
import pandas as pd

primer_cost_model = PrimerCostModel(
    pdf=pd.DataFrame(params['primer_cost']['data'], columns=params['primer_cost']['columns']),
    edf=pd.DataFrame(params['primer_efficiency']['data'], columns=params['primer_efficiency']['columns']),
    time_cost=params['global_time_cost'],
    material_mod=params['global_material_modifier'],
    min_span=params['_primer_min_span'],
    min_anneal=params['primer_min_anneal']
)

In [47]:
synthesis_cost_model = SynthesisCostModel(
    sdf=pd.DataFrame(params['synthesis_cost']['data'], columns=params['synthesis_cost']['columns']),
    primer_cost=primer_cost_model,
    time_cost=params['global_time_cost'],
    material_modifier=params['global_material_modifier'],
    step_size=params['_synthesis_step_size'],
    left_span_range=params['_synthesis_left_span_range']
)

/Users/justinvrana/Documents/GitHub/DASi-DNA-Design/dasi/cost/__init__.py:368: RuntimeWarning: divide by zero encountered in true_divide
  syn_total_cost = (syn_material_cost + syn_time_cost[np.newaxis, ...]) / syn_eff
/Users/justinvrana/Documents/GitHub/DASi-DNA-Design/dasi/cost/__init__.py:368: RuntimeWarning: invalid value encountered in true_divide
  syn_total_cost = (syn_material_cost + syn_time_cost[np.newaxis, ...]) / syn_eff


In [48]:
cost_model = SpanCost(synthesis_cost_model)